In [ ]:
import sqlite3
import pandas as pd

def import_from_sqlite(file):
    cnx = sqlite3.connect(file)
    data = pd.read_sql_query("SELECT * FROM perf_statistics", cnx)
    meta = pd.read_sql_query("SELECT * FROM metadata", cnx, index_col="key")
    cnx.close()
    return meta, data 

def load_data(input_files):
    return [import_from_sqlite(file) for file in input_files]

def plot_timings(results):
    for meta, r in results:
        print(meta.loc['commit_id', 'value'])
        r['iteration_loop_us'].plot(figsize=(20,10), xlabel="iteration", ylabel="time[µs]", legend=True, label=meta.loc['commit_id', 'value'])

    _, res = results[0]
    res['agent_count'].plot(secondary_y=True, ylabel="agents", legend=True)
    
def plot_mean_median(results):
    series = []
    for meta, r in results:
        s = r['iteration_loop_us']
        s.name = meta.loc['commit_id', 'value']
        series.append(s)

    df = pd.DataFrame(series).T
    norms= df.agg(['mean', 'median']).T
    norms.plot()
    

def plot_comparison(inputs):
    data = load_data(inputs)
    plot_timings(data)
    plot_mean_median(data)

# Compare benchmark results

Provide a list of performance statistics in sqlite:

e.g.
```python
versions = ['../master_grosser_stern.sqlite',
            '../older_grosser_stern.sqlite',
           # ... as many as you want to compare ...
           ]
```


In [ ]:
# Add tuples to list with ("id", "path-to-sqlite"),
# e.g. ('master', "master_grosser-stern.sqlite")
to_compare = ['../../benchmarks/1a071cdd_grosser_stern.sqlite',
              '../../benchmarks/7b64c799_grosser_stern.sqlite',
              '../../benchmarks/master_grosser_stern.sqlite',
              '../../benchmarks/grid_grosser_stern.sqlite']
plot_comparison(to_compare)